In [1]:
import deepmatcher as dm
from utils import Hook,return_layer_input_output
import torch
from deepmatcher.data import MatchingIterator

torch.cuda.is_available()

True

In [2]:
hybrid_model = dm.MatchingModel(attr_summarizer='hybrid')
hybrid_model.load_state('models/hybrid_model.pth')
hybrid_model.cuda()

MatchingModel(
  (attr_summarizers): ModuleMap(
    (Song_Name): Hybrid(
      (word_contextualizer): RNN(
        (rnn_groups): ModuleList(
          (0): GRU(300, 150, batch_first=True, bidirectional=True)
        )
        (dropouts): ModuleList(
          (0): Dropout(p=0)
        )
        (bypass_networks): ModuleList(
          (0): None
        )
        (input_dropout): NoMeta(
          (module): Dropout(p=0)
        )
      )
      (word_comparator): Attention(
        (alignment_networks): ModuleList(
          (0): AlignmentNetwork(
            (transform): Transform(
              (transforms): ModuleList(
                (0): Linear(in_features=300, out_features=300, bias=True)
                (1): Linear(in_features=300, out_features=300, bias=True)
              )
              (bypass_networks): ModuleList(
                (0): Bypass(
                  (highway_gate): Linear(in_features=300, out_features=300, bias=True)
                )
                (1): Bypass(


## Datasets creation

In [3]:
attributes = ['Song_Name','Artist_Name','Album_Name','Genre','Price','CopyRight','Time','Released']

In [4]:
negative_dataset = dm.data.process(path='sample_data/itunes-amazon/',train='negative_samples.csv',
                            validation='validation.csv',test='test.csv',cache='ncache.pth')

In [5]:
classifier = hybrid_model.classifier
hookF_classifier = []
hookF_classifier.append(Hook(classifier))

In [6]:
batch_size = 32
splits = MatchingIterator.splits(negative_dataset,batch_size=batch_size)

In [7]:
negative_batches = []
for batch in splits[0]:
    negative_batches.append(batch)

In [8]:
negative_classifier_inputs = []
negative_classifier_outputs = []
for batch in negative_batches:
    classifier_input,classifier_output = return_layer_input_output(hookF_classifier,batch,hybrid_model)
    negative_classifier_inputs.append(classifier_input)
    negative_classifier_outputs.append(classifier_output)

In [9]:
negative_classifier_inputs = list(map(lambda x: x[0][0],negative_classifier_inputs))
negative_classifier_outputs = list(map(lambda x: x[0][0],negative_classifier_outputs))

In [10]:
positive_dataset= dm.data.process(path='sample_data/itunes-amazon/',train='positives_samples.csv',
                            validation='validation.csv',test='test.csv',cache='pcache.pth')

In [11]:
batch_size = 32
splits = MatchingIterator.splits(positive_dataset,batch_size=batch_size)

In [12]:
positive_batches = []
for batch in splits[0]:
    positive_batches.append(batch)

In [13]:
positive_classifier_inputs = []
positive_classifier_outputs = []
for batch in positive_batches:
    classifier_input,classifier_output = return_layer_input_output(hookF_classifier,batch,hybrid_model)
    positive_classifier_inputs.append(classifier_input)
    positive_classifier_outputs.append(classifier_output)

In [14]:
positive_classifier_inputs = list(map(lambda x: x[0][0],positive_classifier_inputs))
positive_classifier_outputs = list(map(lambda x: x[0][0],positive_classifier_outputs))

## Utility methods

## Negative sample analysis

In [20]:
attribute_lenght= len(attributes)

current_sample = 0
#each column of this matrix is related to a specific attribute
negatives_ri_matrix = []
for batch in negative_classifier_inputs:
    for index in range(len(batch)):
        current_sample_ris = []
        for j,attribute in enumerate(attributes):
            print('Processing sample {} with attribute {}'.format(current_sample,attribute))
            it,variation = find_smallest_variation_to_change(hybrid_model.classifier,batch,index,j,1)
            current_sample_ris.append(variation)
        negatives_ri_matrix.append(current_sample_ris)
        current_sample+=1

Processing sample 0 with attribute Song_Name
Processing sample 0 with attribute Artist_Name
Processing sample 0 with attribute Album_Name
Processing sample 0 with attribute Genre
Processing sample 0 with attribute Price
Processing sample 0 with attribute CopyRight
Processing sample 0 with attribute Time
Processing sample 0 with attribute Released
Processing sample 1 with attribute Song_Name
Processing sample 1 with attribute Artist_Name
Processing sample 1 with attribute Album_Name
Processing sample 1 with attribute Genre
Processing sample 1 with attribute Price
Processing sample 1 with attribute CopyRight
Processing sample 1 with attribute Time
Processing sample 1 with attribute Released
Processing sample 2 with attribute Song_Name
Processing sample 2 with attribute Artist_Name
Processing sample 2 with attribute Album_Name
Processing sample 2 with attribute Genre
Processing sample 2 with attribute Price
Processing sample 2 with attribute CopyRight
Processing sample 2 with attribute Ti

Processing sample 23 with attribute Album_Name
Processing sample 23 with attribute Genre
Processing sample 23 with attribute Price
Processing sample 23 with attribute CopyRight
Processing sample 23 with attribute Time
Processing sample 23 with attribute Released
Processing sample 24 with attribute Song_Name
Processing sample 24 with attribute Artist_Name
Processing sample 24 with attribute Album_Name
Processing sample 24 with attribute Genre
Processing sample 24 with attribute Price
Processing sample 24 with attribute CopyRight
Processing sample 24 with attribute Time
Processing sample 24 with attribute Released
Processing sample 25 with attribute Song_Name
Processing sample 25 with attribute Artist_Name
Processing sample 25 with attribute Album_Name
Processing sample 25 with attribute Genre
Processing sample 25 with attribute Price
Processing sample 25 with attribute CopyRight
Processing sample 25 with attribute Time
Processing sample 25 with attribute Released
Processing sample 26 wi

Processing sample 46 with attribute Album_Name
Processing sample 46 with attribute Genre
Processing sample 46 with attribute Price
Processing sample 46 with attribute CopyRight
Processing sample 46 with attribute Time
Processing sample 46 with attribute Released
Processing sample 47 with attribute Song_Name
Processing sample 47 with attribute Artist_Name
Processing sample 47 with attribute Album_Name
Processing sample 47 with attribute Genre
Processing sample 47 with attribute Price
Processing sample 47 with attribute CopyRight
Processing sample 47 with attribute Time
Processing sample 47 with attribute Released
Processing sample 48 with attribute Song_Name
Processing sample 48 with attribute Artist_Name
Processing sample 48 with attribute Album_Name
Processing sample 48 with attribute Genre
Processing sample 48 with attribute Price
Processing sample 48 with attribute CopyRight
Processing sample 48 with attribute Time
Processing sample 48 with attribute Released
Processing sample 49 wi

Processing sample 69 with attribute Album_Name
Processing sample 69 with attribute Genre
Processing sample 69 with attribute Price
Processing sample 69 with attribute CopyRight
Processing sample 69 with attribute Time
Processing sample 69 with attribute Released
Processing sample 70 with attribute Song_Name
Processing sample 70 with attribute Artist_Name
Processing sample 70 with attribute Album_Name
Processing sample 70 with attribute Genre
Processing sample 70 with attribute Price
Processing sample 70 with attribute CopyRight
Processing sample 70 with attribute Time
Processing sample 70 with attribute Released
Processing sample 71 with attribute Song_Name
Processing sample 71 with attribute Artist_Name
Processing sample 71 with attribute Album_Name
Processing sample 71 with attribute Genre
Processing sample 71 with attribute Price
Processing sample 71 with attribute CopyRight
Processing sample 71 with attribute Time
Processing sample 71 with attribute Released
Processing sample 72 wi

Processing sample 92 with attribute Album_Name
Processing sample 92 with attribute Genre
Processing sample 92 with attribute Price
Processing sample 92 with attribute CopyRight
Processing sample 92 with attribute Time
Processing sample 92 with attribute Released
Processing sample 93 with attribute Song_Name
Processing sample 93 with attribute Artist_Name
Processing sample 93 with attribute Album_Name
Processing sample 93 with attribute Genre
Processing sample 93 with attribute Price
Processing sample 93 with attribute CopyRight
Processing sample 93 with attribute Time
Processing sample 93 with attribute Released
Processing sample 94 with attribute Song_Name
Processing sample 94 with attribute Artist_Name
Processing sample 94 with attribute Album_Name
Processing sample 94 with attribute Genre
Processing sample 94 with attribute Price
Processing sample 94 with attribute CopyRight
Processing sample 94 with attribute Time
Processing sample 94 with attribute Released
Processing sample 95 wi

...

Processing sample 295 with attribute Artist_Name
Processing sample 295 with attribute Album_Name
Processing sample 295 with attribute Genre
Processing sample 295 with attribute Price
Processing sample 295 with attribute CopyRight
Processing sample 295 with attribute Time
Processing sample 295 with attribute Released
Processing sample 296 with attribute Song_Name
Processing sample 296 with attribute Artist_Name
Processing sample 296 with attribute Album_Name
Processing sample 296 with attribute Genre
Processing sample 296 with attribute Price
Processing sample 296 with attribute CopyRight
Processing sample 296 with attribute Time
Processing sample 296 with attribute Released
Processing sample 297 with attribute Song_Name
Processing sample 297 with attribute Artist_Name
Processing sample 297 with attribute Album_Name
Processing sample 297 with attribute Genre
Processing sample 297 with attribute Price
Processing sample 297 with attribute CopyRight
Processing sample 297 with attribute Tim

Processing sample 317 with attribute CopyRight
Processing sample 317 with attribute Time
Processing sample 317 with attribute Released
Processing sample 318 with attribute Song_Name
Processing sample 318 with attribute Artist_Name
Processing sample 318 with attribute Album_Name
Processing sample 318 with attribute Genre
Processing sample 318 with attribute Price
Processing sample 318 with attribute CopyRight
Processing sample 318 with attribute Time
Processing sample 318 with attribute Released
Processing sample 319 with attribute Song_Name
Processing sample 319 with attribute Artist_Name
Processing sample 319 with attribute Album_Name
Processing sample 319 with attribute Genre
Processing sample 319 with attribute Price
Processing sample 319 with attribute CopyRight
Processing sample 319 with attribute Time
Processing sample 319 with attribute Released
Processing sample 320 with attribute Song_Name
Processing sample 320 with attribute Artist_Name
Processing sample 320 with attribute Al

Processing sample 340 with attribute Artist_Name
Processing sample 340 with attribute Album_Name
Processing sample 340 with attribute Genre
Processing sample 340 with attribute Price
Processing sample 340 with attribute CopyRight
Processing sample 340 with attribute Time
Processing sample 340 with attribute Released
Processing sample 341 with attribute Song_Name
Processing sample 341 with attribute Artist_Name
Processing sample 341 with attribute Album_Name
Processing sample 341 with attribute Genre
Processing sample 341 with attribute Price
Processing sample 341 with attribute CopyRight
Processing sample 341 with attribute Time
Processing sample 341 with attribute Released
Processing sample 342 with attribute Song_Name
Processing sample 342 with attribute Artist_Name
Processing sample 342 with attribute Album_Name
Processing sample 342 with attribute Genre
Processing sample 342 with attribute Price
Processing sample 342 with attribute CopyRight
Processing sample 342 with attribute Tim

Processing sample 362 with attribute CopyRight
Processing sample 362 with attribute Time
Processing sample 362 with attribute Released
Processing sample 363 with attribute Song_Name
Processing sample 363 with attribute Artist_Name
Processing sample 363 with attribute Album_Name
Processing sample 363 with attribute Genre
Processing sample 363 with attribute Price
Processing sample 363 with attribute CopyRight
Processing sample 363 with attribute Time
Processing sample 363 with attribute Released
Processing sample 364 with attribute Song_Name
Processing sample 364 with attribute Artist_Name
Processing sample 364 with attribute Album_Name
Processing sample 364 with attribute Genre
Processing sample 364 with attribute Price
Processing sample 364 with attribute CopyRight
Processing sample 364 with attribute Time
Processing sample 364 with attribute Released
Processing sample 365 with attribute Song_Name
Processing sample 365 with attribute Artist_Name
Processing sample 365 with attribute Al

Processing sample 385 with attribute Album_Name
Processing sample 385 with attribute Genre
Processing sample 385 with attribute Price
Processing sample 385 with attribute CopyRight
Processing sample 385 with attribute Time
Processing sample 385 with attribute Released
Processing sample 386 with attribute Song_Name
Processing sample 386 with attribute Artist_Name
Processing sample 386 with attribute Album_Name
Processing sample 386 with attribute Genre
Processing sample 386 with attribute Price
Processing sample 386 with attribute CopyRight
Processing sample 386 with attribute Time
Processing sample 386 with attribute Released
Processing sample 387 with attribute Song_Name
Processing sample 387 with attribute Artist_Name
Processing sample 387 with attribute Album_Name
Processing sample 387 with attribute Genre
Processing sample 387 with attribute Price
Processing sample 387 with attribute CopyRight
Processing sample 387 with attribute Time
Processing sample 387 with attribute Released
P

In [21]:
ri_norms_negative_samples = []
for ri_list in negatives_ri_matrix:
    ri_norms_negative_samples.append(list(map(lambda x:torch.norm(x).data[0],ri_list)))

In [22]:
import pandas as pd
variation_df = pd.DataFrame(data= ri_norms_negative_samples,columns=attributes)

In [23]:
variation_df.head()

,Song_Name,Artist_Name,Album_Name,Genre,Price,CopyRight,Time,Released
0,15.023774,17.684738,15.765079,18.447794,13.214149,18.678883,12.342348,19.405939
1,6.134783,9.942346,7.529219,10.315631,7.254350,9.659796,5.731669,9.442733
2,5.292135,10.716656,7.715193,11.211470,5.721026,10.154298,3.680054,9.752934
3,8.246950,10.686390,8.862055,11.020248,9.221317,10.657171,8.808488,10.859727
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [24]:
variation_df.tail()

,Song_Name,Artist_Name,Album_Name,Genre,Price,CopyRight,Time,Released
402,7.254983,12.105650,9.350515,12.631343,8.938286,11.737883,6.735325,11.534370
403,5.159218,9.611754,7.021723,9.907462,5.526068,9.212828,4.215792,8.755407
404,1.516915,3.523539,2.488811,3.548966,1.597985,3.815599,1.295376,2.961972
405,4.075712,11.815557,5.400686,12.194304,4.510251,11.236341,3.556076,6.766049
406,4.102714,7.603525,5.617066,7.835150,4.357930,7.415690,3.296535,6.898834


In [25]:
variation_df.to_csv('experiments-results/negative_samples_ri.csv',index=False)

## Positive samples analisys

In [22]:
attribute_lenght= len(attributes)
positive_samples_ri_matrix = []
current_sample = 0

for batch in positive_classifier_inputs:
    for index in range(len(batch)):
        current_sample_ris = []
        for j,attribute in enumerate(attributes):
            print('Processing sample {} with attribute {}'.format(current_sample,attribute))
            it,variation = find_smallest_variation_to_change(hybrid_model.classifier,batch,index,j,0)
            current_sample_ris.append(variation)
        positive_samples_ri_matrix.append(current_sample_ris)
        current_sample+=1

Processing sample 0 with attribute Song_Name
Processing sample 0 with attribute Artist_Name
Processing sample 0 with attribute Album_Name
Processing sample 0 with attribute Genre
Processing sample 0 with attribute Price
Processing sample 0 with attribute CopyRight
Processing sample 0 with attribute Time
Processing sample 0 with attribute Released
Processing sample 1 with attribute Song_Name
Processing sample 1 with attribute Artist_Name
Processing sample 1 with attribute Album_Name
Processing sample 1 with attribute Genre
Processing sample 1 with attribute Price
Processing sample 1 with attribute CopyRight
Processing sample 1 with attribute Time
Processing sample 1 with attribute Released
Processing sample 2 with attribute Song_Name
Processing sample 2 with attribute Artist_Name
Processing sample 2 with attribute Album_Name
Processing sample 2 with attribute Genre
Processing sample 2 with attribute Price
Processing sample 2 with attribute CopyRight
Processing sample 2 with attribute Ti

Processing sample 23 with attribute Album_Name
Processing sample 23 with attribute Genre
Processing sample 23 with attribute Price
Processing sample 23 with attribute CopyRight
Processing sample 23 with attribute Time
Processing sample 23 with attribute Released
Processing sample 24 with attribute Song_Name
Processing sample 24 with attribute Artist_Name
Processing sample 24 with attribute Album_Name
Processing sample 24 with attribute Genre
Processing sample 24 with attribute Price
Processing sample 24 with attribute CopyRight
Processing sample 24 with attribute Time
Processing sample 24 with attribute Released
Processing sample 25 with attribute Song_Name
Processing sample 25 with attribute Artist_Name
Processing sample 25 with attribute Album_Name
Processing sample 25 with attribute Genre
Processing sample 25 with attribute Price
Processing sample 25 with attribute CopyRight
Processing sample 25 with attribute Time
Processing sample 25 with attribute Released
Processing sample 26 wi

Processing sample 46 with attribute Album_Name
Processing sample 46 with attribute Genre
Processing sample 46 with attribute Price
Processing sample 46 with attribute CopyRight
Processing sample 46 with attribute Time
Processing sample 46 with attribute Released
Processing sample 47 with attribute Song_Name
Processing sample 47 with attribute Artist_Name
Processing sample 47 with attribute Album_Name
Processing sample 47 with attribute Genre
Processing sample 47 with attribute Price
Processing sample 47 with attribute CopyRight
Processing sample 47 with attribute Time
Processing sample 47 with attribute Released
Processing sample 48 with attribute Song_Name
Processing sample 48 with attribute Artist_Name
Processing sample 48 with attribute Album_Name
Processing sample 48 with attribute Genre
Processing sample 48 with attribute Price
Processing sample 48 with attribute CopyRight
Processing sample 48 with attribute Time
Processing sample 48 with attribute Released
Processing sample 49 wi

Processing sample 69 with attribute Album_Name
Processing sample 69 with attribute Genre
Processing sample 69 with attribute Price
Processing sample 69 with attribute CopyRight
Processing sample 69 with attribute Time
Processing sample 69 with attribute Released
Processing sample 70 with attribute Song_Name
Processing sample 70 with attribute Artist_Name
Processing sample 70 with attribute Album_Name
Processing sample 70 with attribute Genre
Processing sample 70 with attribute Price
Processing sample 70 with attribute CopyRight
Processing sample 70 with attribute Time
Processing sample 70 with attribute Released
Processing sample 71 with attribute Song_Name
Processing sample 71 with attribute Artist_Name
Processing sample 71 with attribute Album_Name
Processing sample 71 with attribute Genre
Processing sample 71 with attribute Price
Processing sample 71 with attribute CopyRight
Processing sample 71 with attribute Time
Processing sample 71 with attribute Released
Processing sample 72 wi

Processing sample 92 with attribute Album_Name
Processing sample 92 with attribute Genre
Processing sample 92 with attribute Price
Processing sample 92 with attribute CopyRight
Processing sample 92 with attribute Time
Processing sample 92 with attribute Released
Processing sample 93 with attribute Song_Name
Processing sample 93 with attribute Artist_Name
Processing sample 93 with attribute Album_Name
Processing sample 93 with attribute Genre
Processing sample 93 with attribute Price
Processing sample 93 with attribute CopyRight
Processing sample 93 with attribute Time
Processing sample 93 with attribute Released
Processing sample 94 with attribute Song_Name
Processing sample 94 with attribute Artist_Name
Processing sample 94 with attribute Album_Name
Processing sample 94 with attribute Genre
Processing sample 94 with attribute Price
Processing sample 94 with attribute CopyRight
Processing sample 94 with attribute Time
Processing sample 94 with attribute Released
Processing sample 95 wi

Processing sample 115 with attribute Song_Name
Processing sample 115 with attribute Artist_Name
Processing sample 115 with attribute Album_Name
Processing sample 115 with attribute Genre
Processing sample 115 with attribute Price
Processing sample 115 with attribute CopyRight
Processing sample 115 with attribute Time
Processing sample 115 with attribute Released
Processing sample 116 with attribute Song_Name
Processing sample 116 with attribute Artist_Name
Processing sample 116 with attribute Album_Name
Processing sample 116 with attribute Genre
Processing sample 116 with attribute Price
Processing sample 116 with attribute CopyRight
Processing sample 116 with attribute Time
Processing sample 116 with attribute Released
Processing sample 117 with attribute Song_Name
Processing sample 117 with attribute Artist_Name
Processing sample 117 with attribute Album_Name
Processing sample 117 with attribute Genre
Processing sample 117 with attribute Price
Processing sample 117 with attribute Cop

In [31]:
len(positive_samples_ri_matrix)

132

In [33]:
ri_norms = []
for ri_list in positive_samples_ri_matrix:
    ri_norms.append(list(map(lambda x:torch.norm(x).data[0],ri_list)))

In [34]:
import pandas as pd
variation_df = pd.DataFrame(data= ri_norms,columns=attributes)
variation_df.head(10)

,Song_Name,Artist_Name,Album_Name,Genre,Price,CopyRight,Time,Released
0,7.114510,17.393890,11.948025,17.120317,6.589732,18.535538,5.429970,13.701721
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,8.129299,19.987202,13.563274,20.090685,7.705173,21.586023,6.343812,16.073185
3,12.159317,30.684498,20.136541,31.733618,11.785272,31.356077,10.034542,23.848370
4,6.303537,15.514768,10.768830,15.708781,5.691582,16.876886,4.713202,12.195216
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,5.900886,14.558208,10.391142,15.448649,5.352479,15.948580,4.404889,11.304990
7,4.600801,10.818772,7.275241,10.046092,4.762643,12.164102,3.885354,9.250696
8,8.455050,21.192081,14.321438,22.854610,7.871253,22.649832,6.526680,16.721636
9,1.895168,4.672030,3.180335,4.455643,2.026760,5.100792,1.619094,3.948259


In [35]:
variation_df.to_csv('experiments-results/positive_samples_ri.csv',index=False,sep=';')

In [36]:
variation_df.tail(20)

,Song_Name,Artist_Name,Album_Name,Genre,Price,CopyRight,Time,Released
112,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
113,5.512068,13.499884,9.332538,14.009545,5.029193,14.684381,4.106115,10.556678
114,8.458608,20.781443,14.231000,21.756821,7.852131,21.957876,6.480102,16.431286
115,12.340509,30.199049,19.678627,30.692221,11.810000,30.666946,9.874754,26.884502
116,3.249533,7.647375,5.177475,7.119952,3.325520,8.592142,2.719055,6.532272
117,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
118,5.773713,14.212792,9.882363,14.879112,5.274268,15.404677,4.358005,11.023789
119,5.668377,13.825394,9.804890,14.239173,5.048389,15.452444,4.147325,10.919921
120,4.849197,11.892303,7.886521,11.047002,5.060974,13.120478,4.139968,10.008486
121,8.458608,20.781443,14.231000,21.756821,7.852131,21.957876,6.480102,16.431286


## Finding closer vectors

In [26]:
from scipy.spatial.distance import euclidean
def get_closer_vector(v,batch_list):
    min_distance = 10000000
    i = 0
    best_sample = i
    for batch in batch_list:
        for sample in batch:
            curr_distance = euclidean(v.data,sample.data)
            if curr_distance< min_distance:
                min_distance = curr_distance
                best_sample = i
            i+=1
    return best_sample

In [27]:
def get_closer_vector_on_attribute(v,batch_list,attribute_idx,attribute_lenght):
    min_distance = 1000000
    i = 0
    best_sample = i
    start_index = attribute_idx*attribute_lenght
    end_index = start_index+attribute_lenght
    v_attribute_selected = v[start_index:end_index]
    for batch in batch_list:
        for sample in batch:
            curr_distance = euclidean(v_attribute_selected.data,sample[start_index:end_index].data)
            if curr_distance < min_distance:
                min_distance = curr_distance
                best_sample = i
            i+=1
    return best_sample

In [104]:
#test
closer_sample_index = get_closer_vector_on_attribute(positive_classifier_inputs[0][10],positive_classifier_inputs,0,150)

In [105]:
closer_sample_index

10

In [106]:
idx = get_closer_vector_on_attribute(positive_classifier_inputs[4][5],negative_classifier_inputs,0,150)

In [107]:
idx

377

In [34]:
positive_batches[4].id

['390', '305', '373', '393']

In [28]:
#lista di tuple: vettore più vicino considerando tutti gli elementi e closer solo secondo un attributo
closer_vectors = []
i = 0
for batch in negative_classifier_inputs:
    for sample in batch:
        # lista di coppie in cui il primo elemento indica il campione più vicino considerando tutti gli elementi
        # e il secondo indica il più vicino considerando il singolo attributo
        current_sample_closer_vectors = []
        for j,attribute in enumerate(attributes):
            curr_ri = negatives_ri_matrix[i][j]
            altered_sample = sample + curr_ri
            closer_index = get_closer_vector(altered_sample,positive_classifier_inputs)
            closer_on_attribute = get_closer_vector_on_attribute(altered_sample,
                                                                               positive_classifier_inputs,j,150)
            current_sample_closer_vectors.append((closer_index,closer_on_attribute))
        closer_vectors.append(current_sample_closer_vectors)
        i += 1 

In [108]:
#lista di tuple: vettore piu' vicino, batch index e sample index
positives_closer_vectors = []
i = 0
for batch in positive_classifier_inputs:
    for sample in batch:
        current_sample_closer_vectors = []
        for j,attribute in enumerate(attributes):
            curr_ri = positive_samples_ri_matrix[i][j]
            altered_sample = sample + curr_ri
            closer_index = get_closer_vector(altered_sample,negative_classifier_inputs)
            closer_on_attribute_index = get_closer_vector_on_attribute(altered_sample,
                                                                               negative_classifier_inputs,j,150)
            current_sample_closer_vectors.append((closer_index,closer_on_attribute_index))
        positives_closer_vectors.append(current_sample_closer_vectors)
        i += 1 

In [35]:
ordered_id_positives = []
for batch in positive_batches:
    ordered_id_positives = ordered_id_positives+batch.id

In [36]:
ordered_id_negatives = []
for batch in negative_batches:
    ordered_id_negatives = ordered_id_negatives+batch.id

In [38]:
closer_vectors[1]

[(71, 126),
 (71, 91),
 (71, 126),
 (71, 41),
 (71, 1),
 (71, 54),
 (97, 42),
 (71, 57)]

In [137]:
positives_closer_vectors_df = pd.DataFrame(data= positives_closer_vectors,columns =attributes)

In [138]:
positives_closer_vectors_df.head()

,Song_Name,Artist_Name,Album_Name,Genre,Price,CopyRight,Time,Released
0,"(196, 175)","(196, 238)","(344, 219)","(252, 213)","(344, 8)","(196, 46)","(196, 196)","(344, 281)"
1,"(94, 91)","(94, 342)","(94, 229)","(94, 290)","(94, 25)","(94, 87)","(94, 228)","(94, 369)"
2,"(300, 300)","(105, 94)","(105, 219)","(19, 94)","(105, 8)","(105, 46)","(105, 344)","(105, 385)"
3,"(105, 90)","(105, 302)","(105, 313)","(105, 213)","(105, 8)","(105, 46)","(105, 196)","(105, 281)"
4,"(344, 365)","(344, 238)","(344, 219)","(19, 94)","(344, 8)","(344, 119)","(344, 344)","(344, 281)"


In [39]:
negatives_closer_vectors_df = pd.DataFrame(data = closer_vectors, columns = attributes)

In [41]:
negatives_closer_vectors_df.head()

,Song_Name,Artist_Name,Album_Name,Genre,Price,CopyRight,Time,Released
0,"(71, 91)","(124, 53)","(124, 130)","(124, 117)","(71, 32)","(124, 92)","(124, 40)","(124, 57)"
1,"(71, 126)","(71, 91)","(71, 126)","(71, 41)","(71, 1)","(71, 54)","(97, 42)","(71, 57)"
2,"(124, 126)","(71, 91)","(71, 17)","(124, 41)","(124, 1)","(71, 54)","(97, 42)","(124, 57)"
3,"(130, 97)","(130, 91)","(130, 22)","(130, 61)","(130, 0)","(130, 110)","(130, 25)","(130, 57)"
4,"(64, 20)","(64, 116)","(64, 69)","(64, 66)","(64, 0)","(64, 43)","(64, 42)","(64, 126)"


In [141]:
positives_closer_vectors_df = positives_closer_vectors_df.applymap(lambda c: (ordered_id_negatives[c[0]],
                                                                   ordered_id_negatives[c[1]]))

In [142]:
positives_closer_vectors_df.head()

,Song_Name,Artist_Name,Album_Name,Genre,Price,CopyRight,Time,Released
0,"(357, 180)","(357, 402)","(472, 491)","(241, 401)","(472, 281)","(357, 297)","(357, 357)","(472, 290)"
1,"(422, 81)","(422, 49)","(422, 116)","(422, 371)","(422, 231)","(422, 487)","(422, 380)","(422, 519)"
2,"(188, 188)","(256, 422)","(256, 491)","(418, 422)","(256, 281)","(256, 297)","(256, 472)","(256, 397)"
3,"(256, 441)","(256, 337)","(256, 301)","(256, 401)","(256, 281)","(256, 297)","(256, 357)","(256, 290)"
4,"(472, 426)","(472, 402)","(472, 491)","(418, 422)","(472, 281)","(472, 378)","(472, 472)","(472, 290)"


In [43]:
negatives_closer_vectors_df = negatives_closer_vectors_df.applymap(lambda c: (ordered_id_positives[c[0]],
                                                                   ordered_id_positives[c[1]]))

In [44]:
negatives_closer_vectors_df.head()

,Song_Name,Artist_Name,Album_Name,Genre,Price,CopyRight,Time,Released
0,"(324, 474)","(493, 44)","(493, 373)","(493, 389)","(324, 215)","(493, 253)","(493, 391)","(493, 213)"
1,"(324, 506)","(324, 474)","(324, 506)","(324, 490)","(324, 261)","(324, 254)","(223, 528)","(324, 213)"
2,"(493, 506)","(324, 474)","(324, 114)","(493, 490)","(493, 261)","(324, 254)","(223, 528)","(493, 213)"
3,"(373, 223)","(373, 474)","(373, 478)","(373, 415)","(373, 339)","(373, 408)","(373, 503)","(373, 213)"
4,"(84, 58)","(84, 70)","(84, 495)","(84, 191)","(84, 339)","(84, 250)","(84, 528)","(84, 506)"


In [143]:
positives_closer_vectors_on_attribute_df = positives_closer_vectors_df.applymap(lambda c: c[1])

In [144]:
positives_closer_vectors_on_attribute_df.head()

,Song_Name,Artist_Name,Album_Name,Genre,Price,CopyRight,Time,Released
0,180,402,491,401,281,297,357,290
1,81,49,116,371,231,487,380,519
2,188,422,491,422,281,297,472,397
3,441,337,301,401,281,297,357,290
4,426,402,491,422,281,378,472,290


In [45]:
negatives_closer_vectors_on_attribute_df = negatives_closer_vectors_df.applymap(lambda c : c[1])

In [145]:
positives_closer_vectors_df['Pos_sample_ID']= ordered_id_positives
positives_closer_vectors_on_attribute_df['Pos_sample_ID'] = ordered_id_positives

In [136]:
positives_closer_vectors_df.to_csv('positives_closer_vectors.csv',sep=';',index=False)

In [146]:
positives_closer_vectors_on_attribute_df.to_csv('positives_closer_vectors_on_attribute.csv',sep=';',index=False)

In [46]:
negatives_closer_vectors_df['Neg_sample_ID'] = ordered_id_negatives
negatives_closer_vectors_on_attribute_df['Neg_sample_ID'] = ordered_id_negatives

In [47]:
negatives_closer_vectors_df.to_csv('negatives_closer_vectors.csv',sep=';',index=False)
negatives_closer_vectors_on_attribute_df.to_csv('negatives_closer_vector_on_attribute.csv',sep=';',index=False)

### Some tests

In [38]:
it,variation = find_smallest_variation_to_change(hybrid_model.classifier,positive_classifier_inputs[0],20,6,0)

current loss is 
1.00000e-03 *
  8.9413
 -8.9414
[torch.cuda.FloatTensor of size 2 (GPU 0)]

current gradient norm is Variable containing:
1.00000e-03 *
  6.7240
[torch.cuda.FloatTensor of size 1 (GPU 0)]

current ri norm is Variable containing:
 147.3923
[torch.cuda.FloatTensor of size 1 (GPU 0)]

current loss is 
1.00000e-10 *
  3.9308
  0.0000
[torch.cuda.FloatTensor of size 2 (GPU 0)]

current gradient norm is Variable containing:
1.00000e-10 *
  1.1270
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Moving in wrong direction


In [43]:
positive_classifier_inputs[0][20].data = positive_classifier_inputs[0][20].data.copy_(positive_classifier_inputs[0][20].data+
                                                                                     variation.data)

In [44]:
hybrid_model.classifier.forward(positive_classifier_inputs[0])[20]

Variable containing:
-21.6570
  0.0000
[torch.cuda.FloatTensor of size 2 (GPU 0)]

In [53]:
grads_list = []
h = positive_classifier_inputs[0].register_hook(lambda g: grads_list.append(g))

In [54]:
out = hybrid_model.classifier.forward(positive_classifier_inputs[0])
prob = get_probabilites(out[20])

In [55]:
from torch.autograd import Variable
true_labels = Variable (torch.cuda.FloatTensor([0,1]))